In [22]:
import databento as db
from datetime import datetime
from questdb.ingress import Sender, TimestampNanos
import numpy as np
import os

DATABENTO_API_KEY = os.getenv('DATABENTO_API_KEY', '<YOUR_API_KEY>') 
questdb_conf = "http::addr=host.docker.internal:9000;auto_flush=on;auto_flush_rows=100;auto_flush_interval=1000;"
sender = Sender.from_conf(questdb_conf)
sender.establish()
# First we build up the static data dictionary
instruments = {}

# Enable some basic logging
db.enable_logging("WARN")

# Create a live client and connect
live_client = db.Live(
    key=DATABENTO_API_KEY,
    reconnect_policy="reconnect"
)

# Subscribe to the ohlcv-1s schema for AMZN
live_client.subscribe(
    dataset="EQUS.MINI",
    schema="trades",
    stype_in="raw_symbol",
    #you can comment the next line if you want to get all the stocks
    #this is specially interesting if executing this outside of Nasdaq market hours
    symbols="AMZN,AAPL,MSFT,GOOG,NVDA,META,TSLA,NFLX,ORCL,QSG,BABA",
)


# Create a callback to handle DBN records
def record_callback(record: db.DBNRecord) -> None:    
  if isinstance(record, db.SymbolMappingMsg):     
    instruments.update({record.hd.instrument_id : record.stype_out_symbol})
      
  if isinstance(record, db.TradeMsg):  
    instrument = instruments[record.instrument_id]
    size = record.size
    if record.action == 'A':
        side='ask'
    else:
        side='buy'
    price = record.price * 0.000000001   
    
    sender.row(
      'trades',
    symbols={'symbol': instrument},
    columns={'amount': size,
             'price': price,
             'side': side
            },
             at=TimestampNanos(record.ts_event)
    )

    


# Create a callback to handle reconnections
def reconnect_callback(start, end) -> None:
    sender.flush()
    sender.close()
    sender.establish()
    print(f"reconnection gap from {start} to {end}")


# Create a callback to handle exceptions from `user_callback`
def error_handler(exception: Exception) -> None:
    print(f"an error occurred {exception}")

# We add a print callback to view each record
live_client.add_callback(
    record_callback=record_callback,
    exception_callback=error_handler
)

live_client.add_reconnect_callback(
    reconnect_callback=reconnect_callback,
    exception_callback=error_handler,  # optional error handler
)

# Start the live client to begin data streaming
live_client.start()

# Run the stream for 15 seconds before closing
live_client.block_for_close(timeout=None)

KeyboardInterrupt: 